In [ ]:
import sys
sys.path.append('..')

In [ ]:
from src import models
device = "cuda:0"
mt = models.load_model("gptj", device=device)

For convenience, here's a little API for getting GPT-J completions.

In [ ]:
import torch

@torch.inference_mode()
def complete(prompt, max_new_tokens=1):
    inputs = mt.tokenizer(prompt, return_tensors="pt").to(device)
    outputs = mt.model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        pad_token_id=mt.tokenizer.pad_token_id,
    )
    return mt.tokenizer.batch_decode(outputs)[0]

complete("The Eiffel Tower is located in the city of")

Which attention heads, at layers after the ROME-optimal layer, look at the last subject token?

In [ ]:
from collections import defaultdict

from src.utils import tokenizer_utils

import baukit
from tqdm.auto import tqdm


@torch.inference_mode()
def compute_attns_and_words(prompt_template, subject, layers, ablates=None, subject_token_offset=-1):
    prompt = prompt_template.format(subject)
    si, sj = tokenizer_utils.find_token_range(prompt, subject, tokenizer=mt.tokenizer)
    
    if subject_token_offset < 0:
        ti = sj + subject_token_offset
    else:
        ti = si + subject_token_offset

    z_layernames = [f"transformer.h.{layer}" for layer in layers]
    attn_layernames = [f"transformer.h.{layer}.attn.attn_dropout" for layer in layers]
    
    edit_output = None
    if ablates is not None:
        ablate_heads_by_layer = defaultdict(list)
        for (layer, head) in ablates:
            ablate_layername = f"transformer.h.{layer}.attn.attn_dropout"
            ablate_heads_by_layer[ablate_layername].append(head)

        def edit_output(x, layer):
            if layer not in ablate_heads_by_layer:
                return x
            for head in ablate_heads_by_layer[layer]:
                x[:, head, -1, ti] = 0
            return x

    with baukit.TraceDict(mt.model, (*z_layernames, *attn_layernames), edit_output=edit_output) as ret:
        complete(prompt)

    attns = []
    words = []
    for z_layername, attn_layername in tqdm(tuple(zip(z_layernames, attn_layernames))):
        attn = ret[attn_layername].output[0, :, -1, ti]
        attns.append(attn)

        z = ret[z_layername].output[0][:, -1]
        logits = mt.lm_head(z)
        token_id = logits.argmax(dim=-1).item()
        token_prob = torch.softmax(logits.float(), dim=-1).max()
        word = mt.tokenizer.decode(token_id)
        word = word.strip()
        words.append(f"{word} ({token_prob:.2f})")
    return attns, words

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


subject = "The Space Needle"
prompt_template = "{} is located in the city of"
# subject = "Canberra"
# prompt_template = "{} is the capital city of"
layers = range(27)

attns, words = compute_attns_and_words(
    prompt_template, 
    subject,
    layers,
#     ablates=[(8, 3)],
    subject_token_offset=-1,
)

heads_ordered_list = [
    sorted(enumerate(attn.tolist()), key=lambda kv: kv[-1])
    for attn in attns
]
heads_ordered = [[x[0] for x in heads] for heads in heads_ordered_list]
attns_ordered = [[x[1] for x in heads] for heads in heads_ordered_list]
# attns_ordered = torch.tensor([
#     sorted(attn.tolist())
#     for attn in attns
# ])

sns.heatmap(
    torch.tensor(attns_ordered).numpy(),
    yticklabels=layers,
    cbar=False,
    vmin=0,
    vmax=1,
    # Uncomment this to see the ID.
#     annot=heads_ordered
)
plt.title("Last Token Attn. to Subj Token")
plt.ylabel("Layer")
plt.xlabel("Head")

# plt.yaxis.tick_left()
for i, word in enumerate(words):
    plt.text(attns[0].shape[0] + 0.5, i + 0.5, word, va='center')